## Sparse Grid Regression with the Combination Technique: 


### Function to construct a dataset:

In [ ]:
from sparseSpACE.Function import *
from sparseSpACE.Utils import *
import numpy as np

def construct_dataset(dim, function):
    one_d_grid = np.linspace(0, 1, 50)
    grid_arr = [one_d_grid]*dim
    grid_points = get_cross_product_numpy_array(grid_arr)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

    

### Function to split the dataset into training and test data

In [ ]:
def split_dataset(data, targets):
    training_size = 0
    test_size = 0
    for i in range(len(data)):
        if i%5 != 0:
            training_size += 1
        else:
            test_size += 1
            
    training_data = np.zeros((training_size, len(data[0]))) 
    training_targets = np.zeros(training_size) 
    
    test_data = np.zeros((test_size, len(data[0]))) 
    test_targets = np.zeros(test_size) 
    
    training_index = 0
    test_index = 0
    
    for i in range(len(data)):
        if i%5 != 0:
            training_data[training_index] = data[i]
            training_targets[training_index] = targets[i]
            training_index += 1
        else:
            test_data[test_index] = data[i]
            test_targets[test_index] = targets[i]
            test_index += 1
    
    return training_data, training_targets, test_data, test_targets

### Function to train a combiObject

In [ ]:
def train_regression(training_data, training_targets, regularization, minimum_level, maximum_level):
    dim = len(training_data[0])
    
    a = np.zeros(dim)
    b = np.ones(dim)
    
    operation = Regression(training_data, training_targets, regularization, dim)
    
    combiObject = StandardCombi(a, b, operation=operation)
    
    combiObject.perform_operation(minimum_level, maximum_level)
    
    return operation, combiObject

### Function to test the regression

In [ ]:
import math

def test_regression(test_data, test_targets, combiObject):
    learned_targets = combiObject(test_data)
    
    difference = 0
    for i in range(len(learned_targets)):
        difference += (test_targets[i] - learned_targets[i]) ** 2
        
    return math.sqrt(difference/len(test_targets))

# Result 1: gaussian data set, (size of data set TO error, levels of comp grids TO error)

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

func = GenzGaussian((0.5,0.5), (10,10))
data, target = construct_dataset(2, func)


#split the dataset into training and test data
training_data, training_targets, test_data, test_targets = split_dataset(data, target)   

#initialize the objects with the training data and train them 
operation, combiObject = train_regression(training_data, training_targets, 0.00001, 1, 5)

print("Testfehler:")
print(test_regression(test_data, test_targets, combiObject))


print("Combination Scheme:")
# when you pass the operation the function also plots the contour plot of each component grid
combiObject.print_resulting_combi_scheme()
print("Sparse Grid:")
combiObject.print_resulting_sparsegrid(markersize=20)
#print("Plot of dataset:")
#operation.plot_dataset()
print("Plot of Regression:")
# when contour = True, the contour plot is shown next to the 3D plot
combiObject.plot(contour=True)
#y = combiObject([(0,0), (1,1)])